## Machine Learning Model Lasso L1 and Ridge L2

# Importing the Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Importing the Dataset

In [ ]:
dataset = pd.read_excel('./Real estate valuation data set.xlsx')
dataset.head()
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

# Splitting the Sataset into the Training Set and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 0)

# Training the Simple L1 Regression model on the Training set

In [ ]:
from sklearn import linear_model
L1 = linear_model.Lasso()
L1.fit(X_train,y_train)

# Training the Simple L2 Regression Model on the Training Set

In [ ]:
L2 = linear_model.Ridge()
L2.fit(X_train, y_train)

# Predicting the Test Set Results

In [ ]:
y1_pred = L1.predict(X_test)
y2_pred = L2.predict(X_test)

# Visualising the Training Set Results

In [ ]:
plt.scatter(X_train, y_train, color = 'red')
plt.plot(X_train, L1.predict(X_train), color = 'violet')
plt.title('L1 Lasso Regression (Training Set)')
plt.show()

In [ ]:
plt.scatter(X_train, y_train, color = 'red')
plt.plot(X_train, L2.predict(X_train), color = 'green')
plt.title('L2 Ridge Regression (Training Set)')
plt.show()

# Visualizing the Test Set Results

In [ ]:
plt.scatter(X_test, y_test, color = 'red')
plt.plot(X_test, L1.predict(X_test), color = 'violet')
plt.title('L1 Lass Regressor (Test Set)')
plt.show()

In [ ]:
plt.scatter(X_test, y_test, color = 'red')
plt.plot(X_test, L2.predict(X_test), color = 'green')
plt.title('L2 Ridge Regressor (Test Set)')
plt.show()

## Compare Performance Metrics of L1 and L2

# Mean Absolute Error (MAE)

In [ ]:
print('L1_MAE: ', mean_absolute_error(y_test, y1_pred))
print('L2_MAE: ', mean_absolute_error(y_test, y2_pred))

# Mean Squared Error (MSE)

In [ ]:
print('L1_MSE: ', mean_squared_error(y_test, y1_pred))
print('L2_MSE: ', mean_squared_error(y_test, y2_pred))

# Root Mean Squared Error (RMSE)

In [ ]:
print('L1_RMSE: ', mean_squared_error(y_test, y1_pred, squared=False))
print('L2_RMSE: ', mean_squared_error(y_test, y2_pred, squared=False))

# R-Squared

In [ ]:
print('L1_r2: ', r2_score(y_test, y1_pred))
print('L2_r2: ', r2_score(y_test, y2_pred)

# Adjusted R-Squared

In [ ]:
print('L1_adj_r2: ', (1 - (1-L1_r2)*(len(y)-1)/(len(y) - X.shape[1] - 1)))
print('L2_adj_r2: ', (1 - (1-L2_r2)*(len(y)-1)/(len(y) - X.shape[1] - 1)))